In [49]:
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import mysql # Para ver la versión
import mysql.connector
import pymysql

**CODIGO MYSQL PARA LA COLUMNA**

In [52]:
#DEBIDO A QUE TENEMOS VARIAS TEMPERATURAS MEDIAS POR PROVINCIA, SE HACE UNA QUERY CON LA MEDIA DE TEMPERATURAS EN ESA PROVINCIA POR FECHA.

query = f"""SELECT 
    t1.fecha, 
    AVG(t1.tmed) AS media_tmed, 
    t1.provincia_id, 
    t2.provincia 
FROM 
    valores_climatologicos t1 
RIGHT JOIN 
    provincias t2 ON t1.provincia_id = t2.provincia_id 
GROUP BY 
    t1.fecha, 
    t1.provincia_id, 
    t2.provincia;"""




In [54]:
database = "AEMET"

db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "nueva_contraseña",
                             database = database)

cursor = db.cursor()

cursor.execute(query) # Ejecutamos la query
column_names = cursor.column_names # Nombre de las columnas de la tabla

# Guardamos los datos de la tabla
data = cursor.fetchall()


df = pd.DataFrame(data)

cursor.close()
db.close()

In [55]:
df.rename(columns = {0: "fecha",
                  1: "temperatura",
                  2: "provincia_id",
                    3: "provincia"}, inplace=True)

In [56]:
# Convertir la columna de fecha a tipo datetime (opcional, pero recomendable)
df['fecha'] = pd.to_datetime(df['fecha'])

In [60]:
# Carga el shapefile en un GeoDataFrame
gdf = gpd.read_file("spain-provinces.geojson")

# Crea un mapa españa
centro_espana = [40.4168, -3.7038]
mapa_espana = folium.Map(location=centro_espana, zoom_start=6)
mapa_espana

In [62]:
# Para tener consistencia, cambiamos todas las etiquetas de columnas a tipo string
df_temp["fecha"] = df_temp["fecha"].astype(str)
df_temp["temperatura"] = df_temp["temperatura"].astype(str)
df_temp["provincia"] = df_temp["provincia"].astype(str)
df_temp.isna().sum()/df_temp.shape[0]*100

fecha          0.0
provincia      0.0
temperatura    0.0
dtype: float64

In [73]:
folium.Choropleth(geo_data = gdf,
                  data     = df_temp,
                  columns  = ["temperatura", "provincia"],
                  key_on   = "feature.properties.name").add_to(mapa_espana)

mapa_espana

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''